<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Talleres/Youtube_Detector_Dashboard_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔧 1) Instalar dependencias

In [ ]:
!pip install -q streamlit yt-dlp pydub joblib scikit-learn matplotlib requests ffmpeg
!apt update && apt install -y ffmpeg
!ffmpeg -version
!pip install librosa


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

2) (Opcional) Limpia antiguos directorios

In [ ]:
!rm -rf pages

3) 0_👋_Hello.py completo

In [ ]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import joblib
import io
import os
import re
import traceback
from yt_dlp import YoutubeDL
from pydub import AudioSegment
from sklearn.preprocessing import MinMaxScaler

# --- Configuración de la app ---
st.set_page_config(page_title="Detección de Género Musical", page_icon="🎵")
st.markdown("# Detección de Género Musical 🎵")
st.sidebar.header("Enlace de YouTube o ID (11 caracteres)")

# --- Carga del modelo ---
MODEL_FILE = "knn_genre_detector.pkl"
if not os.path.exists(MODEL_FILE):
    st.error(f"❌ Modelo no encontrado: {MODEL_FILE}")
    st.stop()

@st.cache_data
def load_model():
    return joblib.load(MODEL_FILE)

loaded = load_model()
if isinstance(loaded, dict):
    # extraemos el primer estimador con predict
    model = next((v for v in loaded.values() if hasattr(v, 'predict')), None)
    if model is None:
        st.error("No se encontró un estimador en el pickle.")
        st.stop()
else:
    model = loaded

# --- Función para descargar audio completo ---
@st.cache_data
def download_full_audio(video_id: str) -> AudioSegment:
    # limpia restos previos
    for f in os.listdir():
        if f.startswith("yt_audio."):
            try: os.remove(f)
            except: pass
    opts = {'format': 'bestaudio/best', 'outtmpl': 'yt_audio.%(ext)s', 'quiet': True}
    url = f"https://www.youtube.com/watch?v={video_id}"
    with YoutubeDL(opts) as ydl:
        info = ydl.extract_info(url, download=True)
        ext = info.get('ext', 'm4a')
        fname = f"yt_audio.{ext}"
    audio = AudioSegment.from_file(fname)
    try: os.remove(fname)
    except: pass
    return audio

# --- Interfaz de usuario ---
import librosa

raw = st.sidebar.text_input("Pega URL o ID de YouTube", "").strip()
if raw:
    m = re.search(r"([A-Za-z0-9_-]{11})", raw)
    if not m:
        st.error("❌ ID inválido. Pega URL o ID de 11 caracteres.")
        st.stop()
    vid = m.group(1)

    try:
        full = download_full_audio(vid)
        total_s = int(full.duration_seconds)
        seg_len = min(15, total_s)
        start = max((total_s - seg_len)//2, 0)
        seg = full[start*1000:(start+seg_len)*1000]

        # Exportar segmento a WAV temporal
        tmp_path = 'segment.wav'
        seg.export(tmp_path, format='wav')

        # Reproducción del segmento
        with open(tmp_path, 'rb') as f:
            data = f.read()
        st.subheader(f"Audio segmento: {start}s → {start+seg_len}s")
        st.audio(data, format='audio/wav')

        # Carga con librosa para MFCC
        y, sr = librosa.load(tmp_path, sr=None, mono=True)
        # Extraer MFCC (ejemplo 13 coef.)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        # Promediar cada MFCC en el tiempo
        mfcc_mean = np.mean(mfcc, axis=1)
        # Usar MFCC como features
        X_feat = mfcc_mean

        # Predicción
        st.subheader("Predicción de Género")
        # Ajustar longitud al modelo
        expected = getattr(model, 'n_features_in_', X_feat.size)
        if X_feat.size > expected:
            Xn_adj = X_feat[:expected]
        else:
            Xn_adj = np.pad(X_feat, (0, expected - X_feat.size), 'constant')
        raw_pred = model.predict(Xn_adj.reshape(1, -1))[0]
        label_map = {1:'Reggaeton', 2:'Salsa'}
        gen = label_map.get(raw_pred, f"Clase {raw_pred}")
        st.success(f"**Género previsto:** {gen}")

    except Exception as e:
        st.error(f"🚨 Error procesando: {e}")
        st.text(traceback.format_exc())
else:
    st.info("👉 Ingresa URL o ID en la barra lateral.")

2025-06-16 04:50:23.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:50:23.834 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:50:23.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:50:23.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:50:23.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:50:23.839 No runtime found, using MemoryCacheStorageManager
2025-06-16 04:50:23.845 No runtime found, using MemoryCacheStorageManager
2025-06-16 04:50:23.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:50:23.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

4) Arrancar Streamlit y Cloudflared

In [ ]:
# Mata cualquier Streamlit antiguo
!pkill -f streamlit

# Verifica errores recientes
!tail -n 30 logs.txt || echo "No hay logs.txt aún"





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.81.48.60:8501

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
  Stopping...


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# Ejecutar Streamlit
!streamlit run app.py &> /content/logs.txt &  # o reemplaza app.py por tu script

# Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &


In [ ]:
import time, re, os

time.sleep(5)
url = None
with open('/content/cloudflared.log') as f:
    for line in f:
        # Busca cualquier enlace a trycloudflare.com
        m = re.search(r'https://[\\w\\.-]+\\.trycloudflare\\.com', line)
        if m:
            url = m.group(0)
            break

if url:
    print(f"✅ Tu app está en:\n{url}")
else:
    print("⚠️ No encontré la URL. Revisa cloudflared.log:\n")
    print(open('/content/cloudflared.log').read())

res = input("Escribe '1' para detener Streamlit: ")
if res.strip() == "1":
    os.system("pkill streamlit")
    print("🔴 Streamlit detenido.")



⚠️ No encontré la URL. Revisa cloudflared.log:

2025-06-16T04:50:27Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-06-16T04:50:27Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-06-16T04:50:31Z INF +--------------------------------------------------------------------------------------------+
2025-06-16T04:50:31Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-06-16T04:50:31Z INF |  https